## C S 329E HW 7

# Naive Bayes 

## Your name here (and your partner's name if you are working in a pair)
Parth Dave and Kevin Ly, Group 37

For this week's homework we are going explore one new classification technique:

  - Naive Bayes

We are reusing the version of the Melbourne housing data set from HW6, to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

In addition to building our own Naive Bayes classifier, we are going to compare the performace of our classifier to the [Gaussian Naive Bayes Classifier](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) available in the scikit-learn library. 

In [ ]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
from sklearn.naive_bayes import GaussianNB # The only thing in scikit-learn you can use this assignment

# Starting off loading a training set and setting a variable for the target column, "Type"
df_melb = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/81b236aecee57f6cf65e60afd865d2bb/raw/56ddb53aa90c26ab1bdbfd0b8d8229c8d08ce45a/melb_data_train.csv')
target_col = 'Type'

## Q1 - Fix a column of data to be numeric
If we inspect our dataframe, `df_melb` using the `dtypes` method, we see that the column "Date" is an object.  However, we think this column might contain useful information so we want to convert it to [seconds since epoch](https://en.wikipedia.org/wiki/Unix_time). Use only the exiting imported libraries to create a new column "unixtime". Be careful, the date strings in the file might have some non-uniform formating that you have to fix first.  Print out the min and max epoch time to check your work.  Drop the original "Date" column. 

THESE ARE THE EXACT SAME INSTRUCTIONS FROM HW6! Please take this opportunity to reuse your code (if you got it right last time). 

In [ ]:
# normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format
def standardize_date(d):
    d = d.split('/')
    if len(d[0]) == 1:
      d[0] = '0' + d[0]
    if len(d[1]) == 1:
      d[1] = '0' + d[1]
    if len(d[2]) == 2:
      d[2] = '20' + d[2]
    return '/'.join(d)

In [ ]:
df_melb['Date'] = df_melb['Date'].apply( standardize_date )
df_melb['unixtime'] =  df_melb['Date'].apply(lambda x: calendar.timegm(time.strptime(x, '%d/%m/%Y')))
df_melb = df_melb.drop(columns="Date")

print("The min unixtime is {:d} and the max unixtime is {:d}".format(df_melb['unixtime'].min(),df_melb['unixtime'].max()))

The min unixtime is 1454544000 and the max unixtime is 1506124800


## Q2 Calculating the prior probabilities
Calculate the prior probabilities for each possible "Type" in `df_melb` and populate a dictionary, `dict_priors`, where the key is the possible "Type" values and the value is the prior probabilities. Show the dictionary. Do not hardcode the possible values of "Type".  Don't forget about [value counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html). 

In [ ]:
dict_priors = (df_melb[target_col].value_counts() / len(df_melb)).to_dict()

# show the priors
dict_priors

{'h': 0.452, 't': 0.13, 'u': 0.418}

## Q3 Create a model for the distribution of all of the numeric attributes
For each class, and for each attribute calculate the sample mean and sample standard deviation.  You should store the model in a nested dictionary, `dict_nb_model`, such that `dict_nb_model['h']['Rooms']` is a tuple containing the mean and standard deviation for the target Type 'h' and the attribute 'Rooms'.  Show the model using the `display` function. You should ignore entries that are `NaN` in the mean and [standard deviation](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html) calculation. 

In [ ]:
dict_nb_model = dict()
for target in dict_priors.keys():
    df_target = df_melb[df_melb[target_col] == target]
    dict_nb_model[target] = dict()
    for col in df_target.columns:
        if col != target_col:
            dict_nb_model[target][col] = (df_target[col].mean(skipna=True), df_target[col].std(skipna=True))

In [ ]:
display(dict_nb_model)

{'h': {'Bathroom': (1.5619469026548674, 0.6720871086493075),
  'BuildingArea': (156.24339622641511, 54.62662837301434),
  'Car': (1.7777777777777777, 0.932759177140425),
  'Distance': (12.086725663716809, 7.397501132737295),
  'Landsize': (932.9646017699115, 3830.7934157687164),
  'Postcode': (3103.8982300884954, 98.35750345419703),
  'Price': (1189022.3451327435, 586296.5794417895),
  'Rooms': (3.269911504424779, 0.7258264201127756),
  'YearBuilt': (1954.900826446281, 32.461876347154686),
  'unixtime': (1485717578.761062, 13838562.050601462)},
 't': {'Bathroom': (1.8461538461538463, 0.565430401076506),
  'BuildingArea': (138.66666666666666, 53.498637054290135),
  'Car': (1.6923076923076923, 0.5280588545286915),
  'Distance': (10.766153846153845, 4.870455475462387),
  'Landsize': (268.18461538461537, 276.57700624711265),
  'Postcode': (3121.6153846153848, 100.01588816090862),
  'Price': (1000169.2307692308, 421822.5363389935),
  'Rooms': (2.9076923076923076, 0.6052653582075831),
  'Yea

## Q4 Write a function that calculates the probability of a Gaussian
Given the mean ($\mu$), standard deviation ($\sigma$), and a observed point, `x`, return the probability.  
Use the formula $p(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ ([wiki](https://en.wikipedia.org/wiki/Normal_distribution)).  You should use [numpy's exp](https://numpy.org/doc/stable/reference/generated/numpy.exp.html) function in your solution. 

In [ ]:
def get_p( mu, sigma, x):
    a = 1 / (sigma*np.sqrt(2*np.pi))
    b = -0.5*((x-mu)/sigma)**2
    return a*np.exp(b)

In [ ]:
# Test it
p = get_p( 0, 2, 0.5)
p

0.19333405840142462

## Q5 Write the Naive Bayes classifier function
The Naive Bayes classifier function, `nb_class`, should take as a parameter the prior probability dictionary. `dict_priors`, the dictionary containing all of the gaussian distribution information for each attribue, `dict_nb_model`, and a single observation row (a series generated from iterrows) of the test dataframe. It should return a single target classification. For this problem, all of our attributes are numeric and modeled as Gaussians, so we don't worry about categorical data. Make sure to skip attributes that do not have a value in the observation.  Do not hardcode the possible classification types. 

In [ ]:
def nb_class( dict_priors, dict_nb_model, observation):
    class_cond_probs = dict()
    obs = observation.dropna().drop(target_col)

    for target in dict_priors.keys():
        class_cond_probs[target] = dict_priors[target] * np.prod([get_p(*dict_nb_model[target][col], val) for col, val in obs.iteritems()])

    return max(class_cond_probs, key=class_cond_probs.get)

## Q6 Calculate the accuracy using Naive Bayes classifier function on the test set
Load the test set from file, convert date to unix time and drop the date column, classify each row using your `nb_class`, and then show the accuracy on the test set. 

In [ ]:
df_test = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/c3d53303cebbd986b166591d19254bac/raw/94eb3b2d500d5f7bbc0441a8419cd855349d5d8e/melb_data_test.csv')
df_test['Date'] = df_test['Date'].apply( standardize_date )
df_test['unixtime'] = df_test['Date'].apply(lambda x: calendar.timegm(time.strptime(x, '%d/%m/%Y')))
df_test = df_test.drop(columns="Date")

In [ ]:
predictions = []

for (indx,row) in df_test.iterrows():
    predictions.append(nb_class(dict_priors, dict_nb_model, row))

In [ ]:
acc = sum(df_test[target_col] == predictions)/len(df_test)

In [ ]:
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 57.00%


## Use scikit-learn to do the same thing!

Now we understand the inner workings of the Naive Bayes algorithm, let's compare our results to [scikit-learn's Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) implementation. Use the [GaussianNB](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) to train using the `df_melb`dataframe and test using the `df_test` dataframe. Remember to split `df_melb` into a `df_X` with the numerical attributes, and a `s_y` with the target column. On the `df_melb` frame you will have to fill the empty attributes via imputation since the scikit-learn library can not handle missing values.  Use the same method you used in the last homework (filling the training data with the mean of the non-nan values). 

Answer the following in a markdown cell: do you think imputation hurt or helped the classifier?

In [ ]:
# Imputation training
dict_imputation = dict()
for col in df_melb.columns:
    if col != target_col:
        dict_imputation[col] = df_melb[col].mean(skipna=True)
df_melb.fillna(dict_imputation, inplace=True)
        
# Imputation - apply on the test data
df_test.fillna(dict_imputation, inplace=True)

# Seperate the attributes from the target_col
s_y = df_melb[target_col]
df_X = df_melb.drop(columns=target_col)

In [ ]:
gnb = GaussianNB()
y_pred = gnb.fit(df_X, s_y).predict(df_test.drop(columns=target_col))

In [ ]:
acc = (y_pred == df_test[target_col]).sum() / len(y_pred)
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 37.00%


## ANSWER TO "do you think imputation hurt or helped the classifier?" 
Imputation hurt the classifier because the accuracy was reduced significantly.